# Kredi Kartı Dolandırıcılığı Tespiti

Önceki egzersizler, bir sinir ağının tüm farklı bileşenlerine daha yakından bakmanızı sağladı:
* sıralı (sequential) yoğun (Dense) bir Sinir Ağı mimarisi,
* derleme (compilation) yöntemi,
* modelin eğitilmesi (fitting).

Şimdi **çok fazla veri içeren** gerçek hayat bir veri seti üzerinde çalışalım!

**Veri seti: `Kredi Kartı İşlemleri (Credit Card Transactions)`**

Bu açık uçlu challenge’da, **kredi kartı işlemlerinden elde edilmiş verilerle** çalışacaksınız.

Bu veriler **hassas** olduğu için, toplam 31 sütundan yalnızca 3’ü bilinmektedir; geri kalanlar verileri **anonimleştirmek** amacıyla dönüştürülmüştür (aslında bunlar, orijinal verilerin **PCA projeksiyonlarıdır**).

Bilinen 3 sütun şunlardır:

* `TIME`: İşlemin, veri setindeki ilk işleme göre geçen süresi  
* `AMOUNT`: İşlem tutarı  
* `CLASS` (hedef değişkenimiz):
    * `0 : geçerli işlem`
    * `1 : sahte (fraud) işlem`

❓ **Soru** ❓ Veri setini indirerek başlayın:
* Kaggle üzerinden: [buradan](https://www.kaggle.com/mlg-ulb/creditcardfraud)
* veya bizim URL’mizden: [buradan](https://d32aokrjazspmn.cloudfront.net/materials/creditcard.csv)

Veriyi yükleyerek `X` ve `y` değişkenlerini oluşturun.

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score

from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
df = pd.read_csv("creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## 1. Sınıfların yeniden dengelenmesi

In [3]:
y = df["Class"].astype(int).values
X = df.drop(columns=["Class"]).values

X.shape, y.shape

((284807, 30), (284807,))

☝️ Bu `fraud detection` (sahtekârlık tespiti) challenge’ında **sınıflar aşırı derecede dengesizdir**:
* %99.8 normal işlemler
* %0.2 sahte (fraud) işlemler

**Ciddi yeniden dengeleme (rebalancing) stratejileri uygulamazsak sahtekârlık vakalarını tespit edemeyiz!**

❓ **Soru** ❓
1. **Önce**, veri setinizden üç ayrı bölme oluşturun: `Train / Val / Test`.  
   Doğrulama (validation) ve test setlerinin **dengesiz** kalması son derece önemlidir; böylece modeli değerlendirirken gerçek koşullar korunur ve veri sızıntısı (data leakage) oluşmaz. Test setinizi bu notebook’un **en son hücresine kadar saklayın**!

&nbsp;

2. **İkinci olarak**, yalnızca eğitim setinizi (training set) yeniden dengeleyin. Birçok seçeneğiniz var:

- Azınlık sınıfını rastgele oversample etmek (düz NumPy fonksiyonlarıyla).  
  *(En iyi seçenek değildir; çünkü satırları kopyalayarak veri sızıntısı yaratır.)*
- <a href="https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/">**`Synthetic Minority Oversampling Technique - SMOTE`**</a> kullanarak, mevcut gözlemleri ağırlıklandırıp yeni veri noktaları üretmek
- Buna ek olarak, çoğunluk sınıfını bir miktar küçültmek için  
  <a href="https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/">**`RandomUnderSampler`**</a> da deneyebilirsiniz

In [4]:
# 1) Önce TEST'i ayır
X_tmp, X_test, y_tmp, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

# 2) Sonra VALIDATION'ı ayır (tmp içinden)
X_train, X_val, y_train, y_val = train_test_split(
    X_tmp, y_tmp,
    test_size=0.20,         # tmp'nin %20'si val -> toplamda %16 val olur
    random_state=42,
    stratify=y_tmp
)

print("Shapes:")
print("Train:", X_train.shape, y_train.shape)
print("Val  :", X_val.shape, y_val.shape)
print("Test :", X_test.shape, y_test.shape)

print("\nFraud ratios:")
print("Train:", np.mean(y_train))
print("Val  :", np.mean(y_val))
print("Test :", np.mean(y_test))

Shapes:
Train: (182276, 30) (182276,)
Val  : (45569, 30) (45569,)
Test : (56962, 30) (56962,)

Fraud ratios:
Train: 0.0017281485220215498
Val  : 0.0017336347078057452
Test : 0.0017204452090867595


## 2. Sinir Ağı yinelemeleri

Sınıflarınızı yeniden dengelediğinize göre, test puanınızı optimize etmek için bir sinir ağı uyarlayın. Aşağıdaki ipuçlarını kullanmaktan çekinmeyin:

- Girişlerinizi normalleştirin!
    - Modelinizdeki ön işlemeyi “boru hattı” haline getirmek için tercihen model içinde bir [`Normalization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization) katmanı kullanın. 
    - Veya modelinizin dışında sklearn'in [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) öğesini kullanın, `X_train`, `X_val` ve `X_test` öğelerini uygulayın.
- Modelinizi aşırı uyumlu hale getirin, ardından aşağıdakileri kullanarak düzenleyin:
- Erken Durdurma kriterleri
- [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) katmanları
    - veya [`düzenleyiciler`](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers) katmanları
- 🚨 İzlemek istediğiniz metrikleri ve kullanmak istediğiniz kayıp fonksiyonunu dikkatlice düşünün!

In [5]:
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

print("After SMOTE train distribution:")
print(pd.Series(y_train_bal).value_counts(normalize=True))
print("Balanced train shape:", X_train_bal.shape, y_train_bal.shape)

After SMOTE train distribution:
0    0.5
1    0.5
Name: proportion, dtype: float64
Balanced train shape: (363922, 30) (363922,)


In [6]:
scaler = StandardScaler()
X_train_bal_sc = scaler.fit_transform(X_train_bal)   # FIT burada
X_val_sc       = scaler.transform(X_val)
X_test_sc      = scaler.transform(X_test)

X_train_bal_sc.shape, X_val_sc.shape, X_test_sc.shape

((363922, 30), (45569, 30), (56962, 30))

### 🧪 Kodunu Test Et

Orijinal dengesiz veri kümesinin (`X_test`, `y_test`) temsil edici bir örneğinde gerçek test performansınızın altında kalan sonuçları `precision` ve `recall` değişkenlerine kaydedin.

In [7]:
def build_model(input_dim: int) -> keras.Model:
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.3),
        layers.Dense(32, activation="relu"),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid")  # binary classification
    ])
    return model

model = build_model(X_train_bal_sc.shape[1])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=[
        keras.metrics.Precision(name="precision"),
        keras.metrics.Recall(name="recall"),
        keras.metrics.AUC(curve="PR", name="pr_auc")  # imbalanced datada iyi metrik
    ]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,097 (16.00 KB)

 Trainable params: 4,097 (16.00 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_pr_auc",
        mode="max",
        patience=3,
        restore_best_weights=True
    )
]

history = model.fit(
    X_train_bal_sc, y_train_bal,
    validation_data=(X_val_sc, y_val),
    epochs=30,
    batch_size=2048,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2107 - pr_auc: 0.9795 - precision: 0.9675 - recall: 0.8836 - val_loss: 0.0535 - val_pr_auc: 0.4629 - val_precision: 0.1202 - val_recall: 0.8734
Epoch 2/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0761 - pr_auc: 0.9960 - precision: 0.9820 - recall: 0.9650 - val_loss: 0.0345 - val_pr_auc: 0.6744 - val_precision: 0.1533 - val_recall: 0.8481
Epoch 3/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0500 - pr_auc: 0.9982 - precision: 0.9861 - recall: 0.9795 - val_loss: 0.0291 - val_pr_auc: 0.6792 - val_precision: 0.1775 - val_recall: 0.8608
Epoch 4/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0361 - pr_auc: 0.9989 - precision: 0.9890 - recall: 0.9872 - val_loss: 0.0242 - val_pr_auc: 0.6932 - val_precision: 0.2000 - val_recall: 0.8354
Epoch 5/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0287 - pr_auc: 0.9992 - precision: 0.9908 - recall: 0.9912 - val_loss: 0.0216 - val_pr_auc: 0.6910 - val_precision

In [9]:
y_proba = model.predict(X_test_sc, verbose=0).ravel()
y_pred  = (y_proba >= 0.5).astype(int)

precision = precision_score(y_test, y_pred, zero_division=0)
recall    = recall_score(y_test, y_pred, zero_division=0)

precision, recall

(0.20422535211267606, 0.8877551020408163)

In [10]:
import numpy as np
from sklearn.metrics import precision_score, recall_score

# VAL üzerinde olasılık al
val_proba = model.predict(X_val_sc, verbose=0).ravel()

thresholds = np.linspace(0.01, 0.99, 99)

best_thr = 0.5
best_prec = -1
best_rec = -1

for thr in thresholds:
    val_pred = (val_proba >= thr).astype(int)
    p = precision_score(y_val, val_pred, zero_division=0)
    r = recall_score(y_val, val_pred, zero_division=0)

    # şart: recall en az 0.75 kalsın, precision'u maksimize et
    if r >= 0.75 and p > best_prec:
        best_prec = p
        best_rec = r
        best_thr = thr

print("Best threshold (VAL):", best_thr)
print("VAL precision:", best_prec)
print("VAL recall   :", best_rec)

Best threshold (VAL): 0.99
VAL precision: 0.8051948051948052
VAL recall   : 0.7848101265822784


In [11]:
from nbresult import ChallengeResult
from sklearn.metrics import precision_score, recall_score

test_proba = model.predict(X_test_sc, verbose=0).ravel()
y_pred = (test_proba >= best_thr).astype(int)

precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)

print("TEST precision:", precision)
print("TEST recall   :", recall)

result = ChallengeResult(
    "solution",
    precision=float(precision),
    recall=float(recall),
    fraud_number=int((y_test == 1).sum()),
    non_fraud_number=int((y_test == 0).sum()),
)

result.write()
print(result.check())

TEST precision: 0.7592592592592593
TEST recall   : 0.8367346938775511

============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/gizemtotkanli/.pyenv/versions/3.12.9/envs/workintech_current/bin/python
cachedir: .pytest_cache
rootdir: /Users/gizemtotkanli/code/totkanligizem/S18D3-S-Data-credit-card-challenge/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 2 items

test_solution.py::TestSolution::test_is_score_good_enough PASSED         [ 50%]
test_solution.py::TestSolution::test_is_test_set_representative PASSED   [100%]

============================== 2 passed in 0.00s ===============================


💯 You can commit your code:

git add tests/solution.pickle

git commit -m 'Completed solution step'

git push origin master



## 🏁 İsteğe bağlı: Bu zorluk için Google'ın çözümünü okuyun
Bu oturumdaki tüm zorlukları tamamladığınız için tebrikler!

Son olarak, Google'ın kendi çözümünü doğrudan [Colab'da buradan](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb) okuyun. 

İlginç teknikler ve en iyi uygulamaları keşfedeceksiniz.